In [1]:
# Operações de Crédito - Conta 16000001 (filtragem maiores saldos)

import pandas as pd
import sqlite3

conn = sqlite3.connect('dados/banking.db')

query = """
WITH RankedData AS (
    SELECT
        *,
        ROW_NUMBER() OVER(PARTITION BY data ORDER BY "SALDO" DESC) AS rn
    FROM
        balancetes
    WHERE
        "CONTA" = '16000001'
)
SELECT
    *
FROM
    RankedData
WHERE
    SALDO > 1000000
--    rn <= 10;
"""

model = pd.read_sql_query(query, conn)

conn.close()

model = model[['data', 'cnpj', 'NOME_INSTITUICAO', 'SALDO']]
model['cnpj'] = model['cnpj'].astype(int)
model = model.rename(columns={'SALDO': 'operacoes_de_credito'})


In [2]:
# Receita total
'''
71000008	RECEITAS OPERACIONAIS
73000006	RECEITAS NAO OPERACIONAIS
'''
import numpy as np

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('71000008', '73000006')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)

conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'receita_total'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)

In [3]:
# Ativo total
'''
39999993	TOTAL GERAL DO ATIVO
'''

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('39999993')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)

conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'ativo_total'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [4]:
# Despesa Operacional
'''
81000005	(-) DESPESAS OPERACIONAIS
'''
conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('81000005')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'despesa_operacional'})
table['despesa_operacional'] = -table['despesa_operacional']

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [5]:
# AOC (Average Operating Costs)

model['aoc'] = model['despesa_operacional'] / model['ativo_total']



In [6]:
# Desvio padrão taxas a termo

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    *
FROM
    std_taxas_termo
"""

taxas_termo = pd.read_sql_query(query, conn)

conn.close()

model = pd.merge(
    model,
    taxas_termo,
    on=['data'],
    how='left'
)

In [7]:
# Provisões de crédito
'''
16900008	(-) Provisoes Para Operacoes De Credito
'''

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('16900008')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table['saldo_total_agregado'] = -table['saldo_total_agregado']
table = table.rename(columns={'saldo_total_agregado': 'provisao_credito'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [8]:
# CRERISK - Risco de Crédito

model['crerisk'] = model['provisao_credito'] / model['operacoes_de_credito']


In [9]:
# STD*CRERISK - Interação dos Riscos

model['std_3m*crerisk'] = model['std_3m'] * model['crerisk']
model['std_1a*crerisk'] = model['std_1a'] * model['crerisk']


In [10]:
#Patrimônio Liquido
'''
60000002	PATRIMONIO LIQUIDO
'''
conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('60000002')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'patrimonio_liquido'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)




In [11]:
# RISKAVER

model['riskaver'] = model['patrimonio_liquido'] / model['ativo_total']


In [12]:
# SIZE: Volume de empréstimos concedidos (em logaritmo)

#model['size'] = model['operacoes_de_credito'].diff
model['size'] = np.log10(model['operacoes_de_credito'])


In [13]:
# Receita não financeira
'''
71700009	Rendas De Prestacao De Servicos
71900005	Outras Receitas Operacionais
73000006	RECEITAS NAO OPERACIONAIS
'''
import numpy as np

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
        "CONTA" IN ('71700009', '73000006')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'receita_nao_financeira'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [14]:
# IIP - Implicit Interest Payments

model['iip'] = (model['despesa_operacional'] - model['receita_nao_financeira']) / model['ativo_total']


In [15]:
# Ativos liquidos (Caixa e Equivalentes de Caixa)
'''
Caixa (1.1.1.00.00-9)
Depósitos Bancários (1.1.2.00.00-2)
Reservas Livres (1.1.3.00.00-5)
Aplicações em Operações Compromissadas (1.2.1.00.00-8)
Aplicações em Depósitos Interfinanceiros (1.2.2.00.00-1)
Disponibilidades em Moedas Estrangeiras (1.1.5.00.00-1)
'''

conn = sqlite3.connect('dados/banking.db')

query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('11100009', '11200002', '11300005', '12100008', '12200001', '11500001')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'ativo_liquido'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [16]:
# RESER - Custo de Oportunidade das Reservas

model['reser'] = model['ativo_liquido'] / model['ativo_total']


In [17]:
# Receita Operacional
'''
71000008	RECEITAS OPERACIONAIS
'''
conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('71000008')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'receita_operacional'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)




In [18]:
# EF - Efficiency

model['ef'] = model['despesa_operacional'] / model['receita_operacional']


In [19]:
# Receitas de Juros e Similares (Rendas)
'''
Rendas De Operacoes De Credito (7.1.1.00.00-1)
Rendas de Arrendamento Mercantil (7.1.2.00.00-4)
Rendas de Aplicações Interfinanceiras de Liquidez (7.1.4.00.00-0)
Rendas de Títulos e Valores Mobiliários (7.1.5.00.00-3)
Rendas de Outras Operações Com Características de Crédito (7.1.6.00.00-8)
'''
conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('71100001', '71200004', '71400000', '71500003', '71600008')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'receita_juros'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [20]:
# Despesas de Juros e Encargos (Custos de Captação)
'''
Despesas de captação (81100008)
Despesas de Obrigações por Empréstimos e Repasses (8.1.2.00.00-1)
'''
conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('81100008', '81200001')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'despesa_captacao'})
table['despesa_captacao'] = table['despesa_captacao'] * (-1)

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)

model['log_despesa_captacao'] = np.log10(model['despesa_captacao'])

In [21]:
# Ativos Rentáveis
'''
Operações de Crédito (1.6.0.00.00-1)
Títulos e Valores Mobiliários (1.3.0.00.00-4)
Arrendamento Mercantil (1.7.0.00.00-1)
12000005	APLICACOES INTERFINANCEIRAS DE LIQUIDEZ
'''
conn = sqlite3.connect('dados/banking.db')
query = """
SELECT
    data,
    cnpj,
    SUM("SALDO") AS saldo_total_agregado
FROM
    balancetes
WHERE
    "CONTA" IN ('16000001', '13000004', '17000001', '12000005')
GROUP BY
    data,
    cnpj
ORDER BY
    data,
    cnpj;
"""

table = pd.read_sql_query(query, conn)
conn.close()

table['cnpj'] = (table['cnpj'].astype(str).str.strip().replace('', np.nan).replace('nan', np.nan))
table.dropna(subset=['cnpj'], inplace=True)
table['cnpj'] = table['cnpj'].astype(int)
table = table.rename(columns={'saldo_total_agregado': 'ativos_rentaveis'})

model = pd.merge(
    model,
    table,
    on=['data', 'cnpj'],
    how='left'
)


In [22]:
# NIM

model['nim'] = 100 * (model['receita_juros'] - model['despesa_captacao']) / model['ativos_rentaveis']

In [23]:
# Criando tabela variables

variables = model[['data', 'cnpj', 'NOME_INSTITUICAO', 'nim', 'aoc', 'riskaver','std_3m', 'std_1a', 'crerisk', 'std_3m*crerisk', 'std_1a*crerisk', 'size', 'iip', 'reser', 'ef']]

# Variáveis lags
'''
coluna_grupo = 'cnpj'
colunas_para_verificar = ['nibra', 'nibd', 'ibf', 'lever', 'opc', 'liquid', 'servr', 'mktsh']

for col in colunas_para_verificar:
    variables[f'{col}_lag_2'] = (variables.groupby(coluna_grupo)[col].shift(2))
    variables[f'{col}_lag_1'] = (variables.groupby(coluna_grupo)[col].shift(1))
    variables[f'{col}_lag_3'] = (variables.groupby(coluna_grupo)[col].shift(3))
'''
# Dummies de tempo
dummies_tempo = pd.get_dummies(variables['data'], drop_first=True)
dummies_tempo = dummies_tempo.astype(int)
variables = pd.concat([variables, dummies_tempo], axis=1)

# Removendo linhas com valores nulos, exceto na coluna NOME_INSTITUICAO
todas_colunas = variables.columns.tolist()
colunas_para_verificar = [col for col in todas_colunas if col != 'NOME_INSTITUICAO']
variables = variables.dropna(how='any', subset=colunas_para_verificar)


In [24]:
# Tratando outliers usando o método do IQR (Interquartile Range)
colunas_para_analisar = ['nim', 'aoc', 'riskaver','std_3m', 'std_1a', 'crerisk', 'std_3m*crerisk', 'std_1a*crerisk', 'iip', 'reser', 'ef']

# True significa que a linha *será removida* (outlier).
# Começamos com False, assumindo que nenhuma linha será removida inicialmente.
mascara_outliers_acumulada = pd.Series(False, index=variables.index)

print(f"Tamanho do DataFrame antes da remoção: {len(variables)}")

for col in colunas_para_analisar:
    Q1 = variables[col].quantile(0.10)
    Q3 = variables[col].quantile(0.90)
    IQR = Q3 - Q1

    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    mascara_outliers_coluna = (variables[col] < limite_inferior) | (variables[col] > limite_superior)

    mascara_outliers_acumulada = mascara_outliers_acumulada | mascara_outliers_coluna

    num_outliers_col = mascara_outliers_coluna.sum()
    print(f"Coluna '{col}': {num_outliers_col} outliers identificados (Limites: [{limite_inferior:.2f}, {limite_superior:.2f}])")


# Criamos uma máscara de linhas *a serem mantidas* (True = não é outlier)
mascara_a_manter = ~mascara_outliers_acumulada # O til (~) inverte a máscara
variables_limpo = variables[mascara_a_manter]

print(f"\nTotal de linhas removidas: {mascara_outliers_acumulada.sum()}")
print(f"Tamanho do DataFrame após a remoção: {len(variables_limpo)}")

Tamanho do DataFrame antes da remoção: 135607
Coluna 'nim': 3753 outliers identificados (Limites: [-13.58, 24.69])
Coluna 'aoc': 1270 outliers identificados (Limites: [-0.06, 0.11])
Coluna 'riskaver': 2490 outliers identificados (Limites: [-0.20, 0.40])
Coluna 'std_3m': 847 outliers identificados (Limites: [-0.22, 0.44])
Coluna 'std_1a': 2486 outliers identificados (Limites: [-0.74, 1.86])
Coluna 'crerisk': 4425 outliers identificados (Limites: [-0.16, 0.31])
Coluna 'std_3m*crerisk': 3547 outliers identificados (Limites: [-0.02, 0.04])
Coluna 'std_1a*crerisk': 3922 outliers identificados (Limites: [-0.11, 0.20])
Coluna 'iip': 1428 outliers identificados (Limites: [-0.05, 0.10])
Coluna 'reser': 4759 outliers identificados (Limites: [-0.06, 0.11])
Coluna 'ef': 1752 outliers identificados (Limites: [0.19, 1.59])

Total de linhas removidas: 18555
Tamanho do DataFrame após a remoção: 117052


In [27]:
variables_limpo['aoc'] = 100*variables_limpo['aoc']
variables_limpo['riskaver'] = 100*variables_limpo['riskaver']
variables_limpo['std_3m'] = 100*variables_limpo['std_3m']
variables_limpo['std_1a'] = 100*variables_limpo['std_1a']
variables_limpo['crerisk'] = 100*variables_limpo['crerisk']
variables_limpo['std_3m*crerisk'] = variables_limpo['std_3m']*variables_limpo['crerisk']
variables_limpo['std_1a*crerisk'] = variables_limpo['std_1a']*variables_limpo['crerisk']
variables_limpo['iip'] = 100*variables_limpo['iip']
variables_limpo['reser'] = 100*variables_limpo['reser']
variables_limpo['ef'] = 100*variables_limpo['ef']


/tmp/ipykernel_604208/742725254.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  variables_limpo['aoc'] = 100*variables_limpo['aoc']
/tmp/ipykernel_604208/742725254.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  variables_limpo['riskaver'] = 100*variables_limpo['riskaver']
/tmp/ipykernel_604208/742725254.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

In [29]:
# Regressão
import statsmodels.api as sm

colunas_X_originais = ['aoc', 'riskaver','std_3m', 'std_1a', 'crerisk', 'size', 'iip', 'reser', 'ef']
colunas_X = colunas_X_originais + list(dummies_tempo.columns)
X = variables_limpo[colunas_X]
y = variables_limpo['nim']

stage1 = sm.OLS(y, sm.add_constant(X)).fit()
results_stage1 = stage1
print(results_stage1.summary())

spread_estimado = variables_limpo[['data', 'cnpj', 'NOME_INSTITUICAO']]
spread_estimado['spread_estimado'] = results_stage1.predict()
spread_estimado['spread_puro'] = results_stage1.params['const'] + variables_limpo[dummies_tempo.columns].dot(results_stage1.params[dummies_tempo.columns])


                            OLS Regression Results                            
Dep. Variable:                    nim   R-squared:                       0.421
Model:                            OLS   Adj. R-squared:                  0.420
Method:                 Least Squares   F-statistic:                     464.5
Date:                Sat, 08 Nov 2025   Prob (F-statistic):               0.00
Time:                        18:40:22   Log-Likelihood:            -2.8873e+05
No. Observations:              117052   AIC:                         5.778e+05
Df Residuals:                  116868   BIC:                         5.796e+05
Df Model:                         183                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.7347      0.340      5.108      0.0

/tmp/ipykernel_604208/2463440730.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spread_estimado['spread_estimado'] = results_stage1.predict()
/tmp/ipykernel_604208/2463440730.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spread_estimado['spread_puro'] = results_stage1.params['const'] + variables_limpo[dummies_tempo.columns].dot(results_stage1.params[dummies_tempo.columns])
